# T4. trainer 和 evaluator 的深入介绍

&emsp; 1 &ensp; fastNLP 中的更多 metric 类型

&emsp; &emsp; 1.1 &ensp; 预定义的 metric 类型

&emsp; &emsp; 1.2 &ensp; 自定义的 metric 类型

&emsp; 2 &ensp; fastNLP 中 trainer 的补充介绍
 
&emsp; &emsp; 2.1 &ensp; trainer 的提出构想 

&emsp; &emsp; 2.2 &ensp; trainer 的内部结构

&emsp; &emsp; 2.3 &ensp; 实例：

&emsp; 3 &ensp; fastNLP 中的 driver 与 device

&emsp; &emsp; 3.1 &ensp; driver 的提出构想

&emsp; &emsp; 3.2 &ensp; device 与多卡训练

## 1. fastNLP 中的更多 metric 类型

### 1.1  预定义的 metric 类型

在`fastNLP 0.8`中，除了前几篇`tutorial`中经常见到的**正确率`Accuracy`**，还有其他**预定义的评价标准`metric`**

&emsp; 包括**所有`metric`的基类`Metric`**、适配`Transformers`中相关模型的正确率`TransformersAccuracy`

&emsp; &emsp; **适用于分类语境下的`F1`值`ClassifyFPreRecMetric`**（其中也包括**召回率`Pre`**、**精确率`Rec`**

&emsp; &emsp; **适用于抽取语境下的`F1`值`SpanFPreRecMetric`**；相关基本信息内容见下表，之后是详细分析

| <div align="center">代码名称</div> | <div align="center">简要介绍</div> | <div align="center">代码路径</div> |
|:--|:--|:--|
| `Metric` | 定义`metrics`时继承的基类 | `/core/metrics/metric.py` |
| `Accuracy` | 正确率，最为常用 | `/core/metrics/accuracy.py` |
| `TransformersAccuracy` | 正确率，为了兼容`Transformers`中相关模型 | `/core/metrics/accuracy.py` |
| `ClassifyFPreRecMetric` | 召回率、精确率、F1值，适用于**分类问题** | `/core/metrics/classify_f1_pre_rec_metric.py` |
| `SpanFPreRecMetric` | 召回率、精确率、F1值，适用于**抽取问题** | `/core/metrics/span_f1_pre_rec_metric.py` |

大概的描述一下，给出各个正确率的计算公式

### 2.2  自定义的 metric 类型

在`fastNLP 0.8`中，&emsp; 给一个案例，训练部分留到trainer部分

## 2. fastNLP 中 trainer 的补充介绍

### 2.1  trainer 的提出构想

在`fastNLP 0.8`中，&emsp; 

### 2.2  trainer 的内部结构

在`fastNLP 0.8`中，&emsp; 

'accumulation_steps', 'add_callback_fn', 'backward', 'batch_idx_in_epoch', 'batch_step_fn',
'callback_manager', 'check_batch_step_fn', 'cur_epoch_idx', 'data_device', 'dataloader',
'device', 'driver', 'driver_name', 'epoch_evaluate', 'evaluate_batch_step_fn', 'evaluate_dataloaders',
'evaluate_every', 'evaluate_fn', 'evaluator', 'extract_loss_from_outputs', 'fp16',
'get_no_sync_context', 'global_forward_batches', 'has_checked_train_batch_loop',
'input_mapping', 'kwargs', 'larger_better', 'load_checkpoint', 'load_model', 'marker',
'metrics', 'model', 'model_device', 'monitor', 'move_data_to_device', 'n_epochs', 'num_batches_per_epoch',
'on', 'on_after_backward', 'on_after_optimizers_step', 'on_after_trainer_initialized',
'on_after_zero_grad', 'on_before_backward', 'on_before_optimizers_step', 'on_before_zero_grad',
'on_evaluate_begin', 'on_evaluate_end', 'on_exception', 'on_fetch_data_begin', 'on_fetch_data_end',
'on_load_checkpoint', 'on_load_model', 'on_sanity_check_begin', 'on_sanity_check_end',
'on_save_checkpoint', 'on_save_model', 'on_train_batch_begin', 'on_train_batch_end',
'on_train_begin', 'on_train_end', 'on_train_epoch_begin', 'on_train_epoch_end',
'optimizers', 'output_mapping', 'progress_bar', 'run', 'run_evaluate',
'save_checkpoint', 'save_model', 'start_batch_idx_in_epoch', 'state',
'step', 'step_evaluate', 'total_batches', 'train_batch_loop', 'train_dataloader', 'train_fn', 'train_step',
'trainer_state', 'zero_grad'

&emsp; run(num_train_batch_per_epoch: int = -1, num_eval_batch_per_dl: int = -1, num_eval_sanity_batch: int = 2, resume_from: str = None, resume_training: bool = True, catch_KeyboardInterrupt=None)

### 2.3  实例：

在`fastNLP 0.8`中，&emsp; 

## 3. fastNLP 中的 driver 与 device

### 3.1  driver 的提出构想

在`fastNLP 0.8`中，&emsp; 

### 3.2  device 与多卡训练

在`fastNLP 0.8`中，&emsp; 